# Datagenerering til DigDirCamp 2020

av Bjørn Christian Weinbach, DigDir Campar.

## Bakgrunn

I vår applikasjon skal me demonstrere korleis tilgang til eins eigne persondata kan gjere det enklare å navigere i den norske offentlege sektor, spesielt for ungdom. Denne dataen er ikkje tilgjengeleg vha API i dag og vi er nøydd å generere denne dataen sjølv. Denne jupyter notebooken inneheld python-kode for generering av syntetiske data for ein populasjon som blei brukt som ein del av forarbeidet til prosjektet til DigDir Camparane i 2020.

## Naudsynte imports

In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
import datetime
import calendar
from faker import Faker
fake = Faker('no_NO')

## Globale variablar

$$\text{testpnr} = \text{Test-personnummer}$$

In [ ]:
testpnr = ['23079412414',
           '23079412333',
           '23079412252',
           '23079412171',
           '31010650596',
           '27080350274',
           '04090751315',
           '18060550537',
           '23090350056',
           '23079418366',
           '23079418285',
           '23079418013',
           '23079417815']

## Globale Funksjonar

In [ ]:
def tlfnr():
    """Return phone no in the norwegian style."""
    return random.randint(10000000, 99999999)

def randomName(data):
    """Return random norwegian male or female name."""
    if random.randint(0, 1) == 1:   
        malename = np.array(data["Herrenavn"].sample(1, replace = True))
        malelastname = np.array(data["Etternavn"].sample(1, replace = True))
        return (malename + " " + malelastname)[0], (malename + "." + malelastname + "@norge.no")[0]
    else:
        femalename = np.array(data["Kvinnenavn"].sample(1, replace = True))
        femalelastname = np.array(data["Etternavn"].sample(1, replace = True))
        return (femalename + " " + femalelastname)[0], (femalename + "." + femalelastname + "@norge.no")[0]

def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)

## Last inn datasett

### Kjelder

Norske navn: https://no.wikipedia.org/wiki/Liste_over_norske_mannsnavn , https://no.wikipedia.org/wiki/Liste_over_norske_kvinnenavn og https://no.wikipedia.org/wiki/Liste_over_norske_etternavn

Postnummer: https://data.norge.no/datasets/5e6847ba-156d-4e14-85d3-8d7f8b727523

In [ ]:
data = pd.read_csv('navn.csv')
post = pd.read_csv('postnr.csv', converters={'field_postnummer': lambda x: str(x)}, delimiter=';')
medisiner = pd.read_csv('medisin.csv')
timetype = pd.read_csv('timetype.csv')
klinikk = pd.read_csv('klinikk.csv')
sykehus = pd.read_csv('sykehus.csv')
selskaper = pd.read_csv('selskaper.csv')
emner = pd.read_csv('emner.csv')
vaksinetyper = pd.read_csv('vaksinetyper.csv')

## Kombiner til ein heil populasjon

In [ ]:
titl = []
pnum = []
name = []
email = []
fnum = []
vig = []
adress = []
i = 0
for t in testpnr:
    titl.append("Node " + str(i))
    nam, ema = randomName(data)
    pnum.append(t)
    name.append(nam)
    email.append(ema)
    fnum.append(tlfnr())
    adress.append("Test adresse " + str(random.randint(1, 100)))
    i += 1

populasjon = pd.DataFrame()
populasjon['title'] = titl
populasjon['field_personnummer'] = pnum
populasjon['field_namn'] = name
populasjon['field_epost'] = email
populasjon['field_tlf'] = fnum
populasjon['field_adresse'] = adress
populasjon[post.columns] = pd.concat([post.sample(len(testpnr), replace = True, axis = 0)], axis=0, ignore_index=True)
populasjon.reset_index(drop=True)
populasjon = populasjon.set_index("title")

## Populasjon -> CSV for Drupal import

In [ ]:
populasjon.to_csv('personer.csv')

## VIGO-> CSV for Drupal import

In [ ]:
grades = ['field_rle_karakter',
          'field_naturfag_karakter',
          'field_matematikk_karakter',
          'field_fysikk_',
          'field_kjemi_1_karakter',
          'field_kjemi_2_karakter',
          'field_r_',
          'field_samfunnsfag_karakter',
          'field_spansk_karakter']

absences = ['field_fysikk_fravaer',
            'field_kjemi_1_fravaer',
            'field_kjemi_2_fravaer',
            'field_matematikk_fravaer',
            'field_naturfag_fravaer',
            'field_r2_fravaer',
            'field_rle_fravaer',
            'field_samfunnsfag_fravaer',
            'field_spansk_fravaer']

vigo = pd.DataFrame()
vigo['title'] = titl
vigo['field_personnummer_vigo'] = pnum

for grade in grades:
    vigo[grade] = np.random.randint(1, 6, len(testpnr))

for absence in absences:
    vigo[absence] = abs(np.random.normal(0.08, 0.02, len(testpnr)))

vigo = vigo.reset_index(drop=True)
vigo = vigo.set_index("title")
vigo.to_csv('vigo.csv')

## Helsenorge-fastlege -> CSV for Drupal import

In [ ]:
docnam = []
clinic = []
tlf = []
i = 0
for _ in testpnr:
    tmp2 = np.array(populasjon['field_poststad'])[i] + ' Helsesenter'
    docnam.append(fake.name())
    clinic.append(tmp2)
    tlf.append(tlfnr())
    i += 1

health = pd.DataFrame()
health['title'] = titl
health['field_personnummer_helse'] = pnum
health['field_fastlege'] = docnam
health['field_legesenter'] = clinic
health['field_helse_tlf'] = tlf
health = health.reset_index(drop=True)
healthgp = health.set_index("title")
healthgp.to_csv('helse.csv')

## Helsenorge-resepter -> CSV for drupal import

In [ ]:
res = []
kategori = []
dates = []
node = []
pids = []
i = 0
for pnr in testpnr:
    for i in range(np.random.randint(1, 10)):
        med = medisiner.sample(1, replace = True)
        pids.append(pnr)
        node.append("Node " + str(i))
        res.append(np.array(med["Medisin"])[0])
        kategori.append(np.array(med["Kategori"])[0])
        dates.append(fake.date_between(start_date='-1y', end_date='+1y'))

resept = pd.DataFrame()
resept['title'] = node
resept['field_personnummer_resept'] = pids
resept['field_resept'] = res
resept['field_kategori'] = kategori
resept['field_reseptdato'] = dates
resept = resept.reset_index(drop=True)
resept = resept.set_index("title")
resept.to_csv('resepter.csv')

## Helsenorge-legetime -> CSV for drupal import

In [ ]:
lege = []
dates = []
node = []
pids = []
typ = []
syk = []
kli = []
i = 0
for pnr in testpnr:
    for _ in range(np.random.randint(1, 10)):
        pids.append(pnr)
        node.append("Node " + str(i))
        lege.append(fake.name())
        dates.append(fake.date_time_between(start_date='-1y', end_date='+1y'))
        t = timetype.sample(1, replace = True)
        s = sykehus.sample(1, replace = True)
        k = klinikk.sample(1, replace = True)
        typ.append(np.array(t["type"])[0])
        syk.append(np.array(s["sykehus"])[0])
        kli.append(np.array(k["klinikk"])[0])
        i = i + 1

time = pd.DataFrame()
time['title'] = node
time['field_personnummer_legetime'] = pids
time['field_lege'] = lege
time['field_datetime_legetime'] = dates
time['field_type'] = typ
time['field_sykehus'] = syk
time['field_sted'] = kli
time = time.reset_index(drop=True)
time = time.set_index("title")
time.to_csv('legetime.csv')

## Helsenorge-vaksiner -> CSV for drupal import

In [ ]:
vaksiner = pd.DataFrame()
i = 0
for num in testpnr:
    for _ in range(np.random.randint(7, 15)):
        vaksiner = vaksiner.append( {
            'title' : "Node " + str(i),
            'field_vaksiner_personnummer' : num,
            'field_vaksiner_type' : np.array(vaksinetyper.sample(1, replace=True).reset_index(drop=True))[0][0],
            'field_vaksiner_date' : fake.date_between(start_date='-25y', end_date='today')
            },
            ignore_index=True
        )
        i = i + 1
vaksiner = vaksiner.reset_index(drop=True)
vaksiner = vaksiner.set_index("title")
vaksiner.to_csv('vaksiner.csv')

## Lånekassen -> CSV for Drupal import

Statistikk brukt i dette eksempelet er henta frå https://data.lanekassen.no/statistikk/temasider/videregaaende-utdanning/

In [ ]:
laanekassen = pd.DataFrame()
laanekassen['title'] = titl
laanekassen['field_lanekassen_personnummer'] = pnum
laanekassen['field_utstyrstipend'] = np.ones(len(testpnr)) * 1660
laanekassen['field_stipend'] = np.random.binomial(1, 0.21, len(testpnr)) * 13626
laanekassen['field_borteboerstipend'] = np.random.binomial(1, 0.21, len(testpnr)) * 43753
laanekassen['field_lan'] = np.random.binomial(1, 0.061, len(testpnr)) * 25826
laanekassen['field_lan_utbetaling'] = (laanekassen['field_stipend'] +
                            laanekassen['field_borteboerstipend'] + 
                            laanekassen['field_lan']) * (1/10) + laanekassen['field_utstyrstipend']
laanekassen = laanekassen.reset_index(drop=True)
laanekassen = laanekassen.set_index("title")
laanekassen.to_csv('laanekassen.csv')

## Utbetalinger -> CSV for Drupal import

In [ ]:
utbetalinger = pd.DataFrame()
i = 0
for num in testpnr:
    date = datetime.date(year=2019, month=7, day=12)
    for _ in range(12):
        utbetalinger = utbetalinger.append( {
            'title' : "Node " + str(i),
            'field_utbetalinger_personnummer' : num,
            'field_utbetalinger' : round(np.array(laanekassen[laanekassen['field_lanekassen_personnummer'] == num]['field_lan_utbetaling'])[0]),
            'field_utbetaling_dato' : date
            },
            ignore_index=True
        )
        date = add_months(date, 1)
        i = i + 1
utbetalinger = utbetalinger.reset_index(drop=True)
utbetalinger = utbetalinger.set_index("title")
utbetalinger.to_csv('utbetaling.csv')

## Skatteetaten -> CSV for Drupal import



In [ ]:
skatt = pd.DataFrame()
skatt['title'] = titl
skatt['field_skatt_personnummer'] = pnum
arr1 = np.random.randint(0, 20000, 100)
arr2 = np.random.randint(20000, 100000, 100)
distrobution = np.concatenate((arr1, arr2))
skatt['field_inntekt'] = np.random.choice(distrobution, len(testpnr))
conditions = [
    (skatt['field_inntekt'] <= 55000),
    (skatt['field_inntekt'] > 55000) & (skatt['field_inntekt'] <= 65000),
    (skatt['field_inntekt'] > 65000) & (skatt['field_inntekt'] <= 75000),
    (skatt['field_inntekt'] > 75000) & (skatt['field_inntekt'] <= 85000),
    (skatt['field_inntekt'] > 85000) & (skatt['field_inntekt'] <= 95000)
    ]
values = [0, 
          skatt['field_inntekt']*0.05,
          skatt['field_inntekt']*0.10,
          skatt['field_inntekt']*0.15,
          skatt['field_inntekt']*0.20,]
skatt['field_skatt'] = np.select(conditions, values)
skatt['field_skatteprosent'] = skatt['field_skatt'] / skatt['field_inntekt']
skatt['field_selskap1'] = selskaper.sample(len(testpnr), replace=True).reset_index(drop=True)
skatt['field_dato1'] = [fake.date_between(start_date='-1y', end_date='today') for _ in range(len(testpnr))]
skatt['field_selskap2'] = selskaper.sample(len(testpnr), replace=True).reset_index(drop=True)
skatt['field_dato2'] = [fake.date_between(start_date='-2y', end_date='-1y') for _ in range(len(testpnr))]
skatt['field_selskap3'] = selskaper.sample(len(testpnr), replace=True).reset_index(drop=True)
skatt['field_dato3'] = [fake.date_between(start_date='-3y', end_date='-2y') for _ in range(len(testpnr))]
skatt = skatt.reset_index(drop=True)
skatt = skatt.set_index("title")
skatt.to_csv('skatt.csv')

## Digipost -> CSV for drupal import

In [ ]:
digipost = pd.DataFrame()
i = 0
for num in testpnr:
    for _ in range(np.random.randint(1, 10)):
        digipost = digipost.append( {
            'title' : "Node " + str(i),
            'field_digipost_personnummer' : num,
            'field_digipost_emne' : np.array(emner.sample(1, replace=True).reset_index(drop=True))[0][0],
            'field_digipost_innhold' : fake.text(1000),
            'field_digipost_datetime' : fake.date_between(start_date='-8y', end_date='today')
            },
            ignore_index=True
        )
        i = i + 1
digipost['field_digipost_sender'] = selskaper.sample(i-1, replace=True).reset_index(drop=True)
digipost = digipost.reset_index(drop=True)
digipost = digipost.set_index("title")
digipost.to_csv('digipost.csv')